In [73]:
!pip install tensorflow



In [74]:
import os
import shutil
import cv2
import pydicom
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tkinter as tk
from tkinter import filedialog
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [75]:
input_chest_folder = "F:/images/chest"
input_nonchest_folder = "F:/images/others"
output_chest_folder = "F:/images/preprocessed_data/chest"
output_nonchest_folder = "F:/images/preprocessed_data/non_chest"
image_size = 224  # Resize to 224x224 
convert_to_rgb = True #for DenseNet

#Delete/Create Output Folders
for folder in [output_chest_folder, output_nonchest_folder]:
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder, exist_ok=True)

def preprocess_image(img_path):
    try:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise Exception("Could not read image")

        # Normalize to [0, 255]
        img = (img - np.min(img)) / (np.max(img) - np.min(img)) * 255.0
        img = img.astype(np.uint8)

        # Resize
        img_resized = cv2.resize(img, (image_size, image_size))

        # Convert to RGB 
        if convert_to_rgb:
            img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)

        return img_resized

    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None

def process_folder(input_folder, output_folder, prefix):
    files = [f for f in os.listdir(input_folder) if f.lower().endswith((".png", ".jpg", ".jpeg"))]
    files.sort()  #sort for consistent naming
    for idx, file in enumerate(tqdm(files, desc=f"Processing {prefix}")):
        input_path = os.path.join(input_folder, file)
        processed_img = preprocess_image(input_path)
        if processed_img is not None:
            new_filename = f"{prefix}_{idx + 1:04d}.png"
            output_path = os.path.join(output_folder, new_filename)
            cv2.imwrite(output_path, processed_img)

#Process Both Folders
process_folder(input_chest_folder, output_chest_folder, "chest")
process_folder(input_nonchest_folder, output_nonchest_folder, "others")

print("\nAll images preprocessed and saved.")


Processing others: 100%|█████████████████████████████████████████████████████████████| 393/393 [00:11<00:00, 34.79it/s]


All images preprocessed and saved.


In [76]:
base_path = "F:/images/preprocessed_data"  # Base folder with 'chest' and 'others' subfolders
folders = ['chest', 'non_chest']
num_augmentations = 4
image_size = (224, 224)  

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=(0.9, 1.1),
    horizontal_flip=True,
    fill_mode='nearest'
)

def augment_image(image_path, save_prefix, output_folder, num_copies):
    img = load_img(image_path, color_mode='rgb' if convert_to_rgb else 'grayscale', target_size=image_size)
    x = img_to_array(img).reshape((1,) + img_to_array(img).shape)
    gen = datagen.flow(x, batch_size=1)
    
    for i in range(1, num_copies + 1):
        suffix = f"_aug{str(i).zfill(3)}"
        filename = f"{save_prefix}{suffix}.png"
        save_path = os.path.join(output_folder, filename)
        aug_img_array = next(gen)[0].astype('uint8')
        img_pil = Image.fromarray(aug_img_array if convert_to_rgb else aug_img_array.squeeze(), mode='RGB' if convert_to_rgb else 'L')
        img_pil.save(save_path)

#Augmentation Process
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    augment_count = defaultdict(int)
    
    print(f"\nAugmenting images in folder: {folder_path}")
    
    for filename in tqdm(os.listdir(folder_path), desc=f"Augmenting {folder}"):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        
        src_path = os.path.join(folder_path, filename)
        base_name = os.path.splitext(filename)[0]
        new_base = f"{folder}_{base_name}"
        
        # Augment and save augmented images in the same folder
        augment_image(src_path, new_base, folder_path, num_augmentations)
        
        augment_count[filename] = num_augmentations
    
    print(f"Completed augmentation of {len(augment_count)} images in '{folder}' with {num_augmentations} augmentations each.")

print("\nData augmentation completed for both 'chest' and 'others' folders")



Augmenting images in folder: F:/images/preprocessed_data\chest


Augmenting chest: 100%|██████████████████████████████████████████████████████████████| 356/356 [00:32<00:00, 10.85it/s]


Completed augmentation of 356 images in 'chest' with 4 augmentations each.

Augmenting images in folder: F:/images/preprocessed_data\non_chest


Augmenting non_chest: 100%|██████████████████████████████████████████████████████████| 393/393 [00:33<00:00, 11.59it/s]

Completed augmentation of 393 images in 'non_chest' with 4 augmentations each.

Data augmentation completed for both 'chest' and 'others' folders


In [77]:
batch_size = 16
num_classes = 2
learning_rate = 1e-4 #how much the model's weights are updated
epochs = 10
img_size = (224, 224)

In [78]:
#Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,       # 20% data for validation
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [79]:
# Train generator loads from subfolders ('chest', 'others')
train_generator = train_datagen.flow_from_directory(
    base_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',         #returns integer labels (0 or 1)
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    base_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation',
    shuffle=False,
    seed=42
)


Found 2996 images belonging to 2 classes.
Found 749 images belonging to 2 classes.


In [80]:
#Model Definition
input_shape = img_size + (3,)  

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

#Freeze base layers initially to retain pre-trained knowledge from ImageNet.
for layer in base_model.layers:
    layer.trainable = False
    
#Helps prevent overfitting.
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [81]:
#Training
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    verbose=1
)

C:\Users\intmay25\AppData\Local\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 185s 931ms/step - accuracy: 0.5322 - loss: 0.9825 - val_accuracy: 0.8398 - val_loss: 0.4135
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 168s 894ms/step - accuracy: 0.8052 - loss: 0.4255 - val_accuracy: 0.9947 - val_loss: 0.1473
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 168s 896ms/step - accuracy: 0.8985 - loss: 0.2402 - val_accuracy: 0.9973 - val_loss: 0.0740
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 167s 887ms/step - accuracy: 0.9450 - loss: 0.1572 - val_accuracy: 0.9960 - val_loss: 0.0460
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 170s 905ms/step - accuracy: 0.9737 - loss: 0.1051 - val_accuracy: 0.9973 - val_loss: 0.0325
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 165s 876ms/step - accuracy: 0.9673 - loss: 0.1032 - val_accuracy: 0.9973 - val_loss: 0.0250
Epoch 7/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 172s 914ms/step - accuracy: 0.9774 - loss: 0.0813 - val_accuracy: 0.9973 - val_loss: 0.0182
Epoch 8/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 172s 914ms/step - accuracy: 0.9799 -

In [82]:
#After training completes
print("\nEvaluation on Training Data")
train_loss, train_acc = model.evaluate(train_generator, verbose=1)
print(f"Training Accuracy: {train_acc*100:.2f}%")

print("\nEvaluation on Validation Data")
val_loss, val_acc = model.evaluate(validation_generator, verbose=1)
print(f"Validation Accuracy: {val_acc*100:.2f}%")


Evaluation on Training Data
188/188 ━━━━━━━━━━━━━━━━━━━━ 137s 729ms/step - accuracy: 0.9978 - loss: 0.0201
Training Accuracy: 99.63%

Evaluation on Validation Data
47/47 ━━━━━━━━━━━━━━━━━━━━ 34s 710ms/step - accuracy: 0.9955 - loss: 0.0170
Validation Accuracy: 99.87%


In [83]:
#Get predictions from validation generator
val_preds = model.predict(validation_generator)
y_pred = np.argmax(val_preds, axis=1)
y_true = validation_generator.classes

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=validation_generator.class_indices.keys()))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

47/47 ━━━━━━━━━━━━━━━━━━━━ 40s 787ms/step

Classification Report:
              precision    recall  f1-score   support

       chest       1.00      1.00      1.00       356
   non_chest       1.00      1.00      1.00       393

    accuracy                           1.00       749
   macro avg       1.00      1.00      1.00       749
weighted avg       1.00      1.00      1.00       749


Confusion Matrix:
[[355   1]
 [  0 393]]


In [84]:
model.save("chest_xray_classifier1.h5")
print(" Model saved successfully!")

 Model saved successfully!


In [85]:
#Load your trained model
model = load_model("chest_xray_classifier1.h5")

In [86]:
# Define your class labels
class_labels = ['chest', 'others']  # Modify based on your model

def crop_lung_region(img):
    h, w = img.shape
    x_start = int(w * 0.15)
    x_end = int(w * 0.85)
    y_start = int(h * 0.05)
    y_end = int(h * 0.85)
    return img[y_start:y_end, x_start:x_end]

def load_image(path, target_size=(224, 224)):
    ext = os.path.splitext(path)[1].lower()
    if ext == '.dcm':
        dcm = pydicom.dcmread(path)
        img = dcm.pixel_array.astype(np.float32)
        img = (img - np.min(img)) / (np.max(img) - np.min(img)) * 255.0
        img = img.astype(np.uint8)
        img_cropped = crop_lung_region(img)
        img_rgb = cv2.cvtColor(img_cropped, cv2.COLOR_GRAY2RGB)
        img_resized = cv2.resize(img_rgb, target_size)
        return img_resized
    else:
        img = image.load_img(path, target_size=target_size)
        img = image.img_to_array(img)
        return img.astype(np.uint8)


In [87]:
def predict_folder_images():
    root = tk.Tk()
    root.withdraw()

    folder_path = filedialog.askdirectory(title="Select Folder with X-ray Images")
    if not folder_path:
        print("No folder selected.")
        return

    results = []
    supported_exts = ('.png', '.jpg', '.jpeg', '.bmp', '.dcm')

    for fname in os.listdir(folder_path):
        if not fname.lower().endswith(supported_exts):
            continue

        full_path = os.path.join(folder_path, fname)
        try:
            img = load_image(full_path)
            img_array = np.expand_dims(img, axis=0) / 255.0

            prediction = model.predict(img_array)
            predicted_class = class_labels[np.argmax(prediction)]
            confidence = np.max(prediction) * 100

            results.append({
                'File Name': fname,
                'Predicted Class': predicted_class,
                'Confidence (%)': round(confidence, 2)
            })

            print(f"{fname}: {predicted_class} ({confidence:.2f}%)")

        except Exception as e:
            print(f"Error processing {fname}: {e}")

    if results:
        df = pd.DataFrame(results)

        # Ask where to save the CSV file
        save_path = filedialog.asksaveasfilename(
            defaultextension='.csv',
            filetypes=[("CSV files", "*.csv")],
            title="Save Predictions CSV"
        )

        if save_path:
            df.to_csv(save_path, index=False)
            print(f"\nPredictions saved to: {save_path}")
        else:
            print("\nSave cancelled.")

    else:
        print("No valid images found or predictions failed.")


In [88]:
# Run the folder-based prediction
predict_folder_images()


C:\Users\intmay25\AppData\Local\anaconda3\Lib\site-packages\pydicom\pixels\utils.py:222: UserWarning: A value of 'None' for (0028,0008) 'Number of Frames' is invalid, assuming 1 frame
  warn_and_log(


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


NIRT_DS01_00001.dcm: chest (97.58%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step


NIRT_DS01_00002.dcm: chest (99.76%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step


NIRT_DS01_00003.dcm: chest (99.77%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step


...

NIRT_DS01_00195.dcm: chest (90.80%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step


NIRT_DS01_00196.dcm: chest (98.53%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step


NIRT_DS01_00197.dcm: chest (99.43%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step


NIRT_DS01_00198.dcm: chest (99.43%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step


NIRT_DS01_00199.dcm: chest (98.87%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
NIRT_DS01_00200.dcm: chest (99.47%)

Predictions saved to: F:/set2_pred7.csv
